# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

---
## Step 0: Load The Data

In [ ]:
import cv2
from sklearn.utils import shuffle
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Load pickled data
import pickle

training_file = "./dataset/train.p"
validation_file= "./dataset/valid.p"
testing_file = "./dataset/test.p"

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(validation_file, mode='rb') as f:
    valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_valid, y_valid = valid['features'], valid['labels']
X_test, y_test = test['features'], test['labels']

---

## Step 1: Dataset Summary & Exploration

The pickled data is a dictionary with 4 key/value pairs:

- `'features'` is a 4D array containing raw pixel data of the traffic sign images, (num examples, width, height, channels).
- `'labels'` is a 1D array containing the label/class id of the traffic sign. The file `signnames.csv` contains id -> name mappings for each id.
- `'sizes'` is a list containing tuples, (width, height) representing the original width and height the image.
- `'coords'` is a list containing tuples, (x1, y1, x2, y2) representing coordinates of a bounding box around the sign in the image. **THESE COORDINATES ASSUME THE ORIGINAL IMAGE. THE PICKLED DATA CONTAINS RESIZED VERSIONS (32 by 32) OF THESE IMAGES**

### Provide a Basic Summary of the Data Set Using Python, Numpy and/or Pandas

In [ ]:
n_train = X_train.shape[0]
n_test = X_test.shape[0]
image_shape = X_train[0].shape
n_classes = np.unique(y_train).shape[0]

print("Number of training examples =", n_train)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

### Include an exploratory visualization of the dataset

In [ ]:
plt.hist(y_train, n_classes, histtype='bar')
plt.title("Classes Are Very Unevenly Distributed")
plt.xlabel("Traffic Sign Class")
plt.ylabel("Examples in Training Set")
plt.show()

In [ ]:
# load human readable class labels from csv and store in a dict
import csv
class_labels = {}
with open('signnames.csv', mode='r') as infile:
    reader = csv.reader(infile)
    class_labels = {key: val for key, val in reader}

In [ ]:
def visualize_class(X_data, y_labels=[], cmap=None,
                    rows=1, cols=12, maxtitlelen=18, figsize=(15,1)):
    """
    Convenience method to visualize sets of images. If a set of corresponding
    class labels is passed, they will be printed along with the image.
    """
    has_labels = len(y_labels) > 0
    assert not has_labels or len(X_data) == len(y_labels)

    plt.figure(figsize=figsize)
    for i, img in enumerate(X_data):
        plt.subplot(rows, cols, i+1, xticks=[], yticks=[])
        plt.imshow(img, interpolation="nearest", cmap=cmap)
        if has_labels:
            y_val = y_labels[i]
            y_label = class_labels[str(y_val)][:maxtitlelen]
            plt.title("{}: {}".format(y_val, y_label), loc="left")
    plt.show()

In [ ]:
for cls in range(n_classes):
    print("=" * 95)
    print('LABEL: {} "{}"'.format(cls, class_labels[str(cls)]))
    print('COUNT: {}'.format(y_train[y_train==cls].shape[0]))
    class_xs = X_train[y_train==cls]
    visualize_class(class_xs[np.random.choice(class_xs.shape[0], size=12)])

----

## Step 2: Design and Test a Model Architecture

### Pre-process the Data Set (normalization, grayscale, etc.)

In [ ]:
def preprocess(images):
    """
    1. Convert each image to grayscale and equalize histogram
    2. Normalize image values and set the mean to zero
    """
    out = np.empty(images.shape[:3], dtype=np.float32)
    for idx in range(out.shape[0]):
        out[idx] = cv2.equalizeHist(cv2.cvtColor(images[idx], cv2.COLOR_RGB2GRAY))
        out[idx] = cv2.normalize(out[idx], out[idx], alpha=-0.5, beta=0.5,
                                 norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    return out.reshape(out.shape + (1,))

In [ ]:
# preprocess data and copy y_labels to avoid shuffling original data during training
X_train_process, y_train_process = preprocess(X_train), np.copy(y_train)
X_valid_process, y_valid_process = preprocess(X_valid), np.copy(y_valid)
X_test_process, y_test_process = preprocess(X_test), np.copy(y_test)

In [ ]:
for index in range(3):
    rand_indices = np.random.choice(X_train.shape[0], size=12)
    subset_unprocessed = X_train[rand_indices]
    subset_processed = X_train_process[rand_indices]
    visualize_class(subset_unprocessed)
    visualize_class(subset_processed.reshape(subset_processed.shape[:3]), cmap="gray")

### Model Architecture

In [ ]:
import tensorflow as tf
from tensorflow.contrib.layers import flatten

EPOCHS = 101
BATCH_SIZE = 256
LEARN_RATE = 0.001
SAVE_FILE = "./dataset/model.ckpt"

# Store layers weight & bias
weight_var = lambda w_shape: tf.Variable(tf.truncated_normal(w_shape, mean=0, stddev=0.1), name="weights")
bias_var = lambda n_vars: tf.Variable(tf.zeros(n_vars), name="biases")

def conv2d(layer, W, b, strides=1):
    out = tf.nn.conv2d(layer, W, strides=(1,strides,strides,1), padding="SAME")
    out = tf.nn.bias_add(out, b)
    return tf.nn.relu(out, name="relu")

def pool(layer, k=2):
    return tf.nn.max_pool(layer, ksize=(1,k,k,1), strides=(1,k,k,1), padding="SAME", name="pool")

def fully_connected(layer, W, b, activate=True):
    out = tf.add(tf.matmul(layer, W), b)
    if activate:
        out = tf.nn.relu(out, name="relu")
    return out

In [ ]:
keep_prob = tf.placeholder(tf.float32)

def Model(X_inputs):
    # Layer 1: Convolutional. Input = 32x32x1.
    with tf.variable_scope("conv1"):
        conv1 = conv2d(X_inputs, weight_var((5,5,1,32)), bias_var(32)) # Output = 32x32x32.
        conv1 = pool(conv1, k=2) # Output = 16x16x32.

    # Layer 2: Convolutional.
    with tf.variable_scope("conv2"):
        conv2 = conv2d(conv1, weight_var((5,5,32,64)), bias_var(64)) # Output = 16x16x64.
        conv2 = pool(conv2, k=2) # Output = 8x8x64.
    
    # Layer 3: Convolutional.
    with tf.variable_scope("conv3"):
        conv3 = conv2d(conv2, weight_var((5,5,64,128)), bias_var(128)) # Output = 8x8x128.
        conv3 = pool(conv3, k=2) # Output = 4x4x128.

    # Layer 4: Flatten, Pool & Concatenate Layers.
    with tf.variable_scope("flat"):
        try:
            # TensorFlow v1.0.1: tf.concat(values, axis);
            flat = tf.concat((flatten(pool(conv1, k=4)),
                              flatten(pool(conv2, k=2)),
                              flatten(conv3)), 1) # Output 3584.
        # Prior TF versions: tf.concat(axis, values) :D
        except:
            flat = tf.concat(1, (flatten(pool(conv1, k=4)),
                              flatten(pool(conv2, k=2)),
                              flatten(conv3))) # Output 3584.
        flat = tf.nn.dropout(flat, keep_prob=keep_prob)

    # Layer 5: Fully Connected.
    with tf.variable_scope("fc1"):
        fc1 = fully_connected(flat, weight_var((3584,1024)), bias_var(1024)) # Output = 1024.
        fc1 = tf.nn.dropout(fc1, keep_prob=keep_prob)

    # Layer 6: Fully Connected. Out.
    with tf.variable_scope("fc2"):
        return fully_connected(fc1, weight_var((1024,43)), bias_var(43), activate=False) # Output = 43.

### Train, Validate and Test the Model

A validation set can be used to assess how well the model is performing. A low accuracy on the training and validation
sets imply underfitting. A high accuracy on the training set but low accuracy on the validation set implies overfitting.

## Features and Labels

In [ ]:
X_vals = tf.placeholder(tf.float32, (None, 32, 32, 1))
y_vals = tf.placeholder(tf.int32, (None))
one_hot_y = tf.one_hot(y_vals, 43)

## Training Pipeline

In [ ]:
logits = Model(X_vals)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=one_hot_y)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate=LEARN_RATE)
training_operation = optimizer.minimize(loss_operation)

## Model Evaluation

In [ ]:
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

def evaluate(X_data, y_labels):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_labels[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={X_vals: batch_x, y_vals: batch_y, keep_prob: 1.})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

## Train the Model

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = len(X_train_process)
    for i in range(EPOCHS):
        X_train_process, y_train_process = shuffle(X_train_process, y_train_process)
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train_process[offset:end], y_train_process[offset:end]
            sess.run(training_operation, feed_dict={X_vals: batch_x, y_vals: batch_y, keep_prob: 0.50})
        validation_accuracy = evaluate(X_valid_process, y_valid_process)
        if i % 10 == 0:
            print("EPOCH {}: Validation Accuracy = {:.3f}".format(i, validation_accuracy))
    saver.save(sess, SAVE_FILE)
    print("Model saved")

## Evaluate the Model

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    tf.train.Saver().restore(sess, SAVE_FILE)
    test_accuracy = evaluate(X_test_process, y_test_process)
    print("Test Accuracy = {:.3f}".format(test_accuracy))

---

## Step 3: Test a Model on New Images

### Load and Output the Images

In [ ]:
sign_classes = np.array([7, 9, 11, 17, 25, 31, 40])
n_signs = len(sign_classes)
sign_data = np.empty((n_signs,32,32,3), dtype=np.uint8)
sign_path = "./signs/{}.jpg"

for i, label in enumerate(sign_classes):
    sign_data[i] = cv2.cvtColor(cv2.imread(sign_path.format(label), cv2.IMREAD_COLOR),
                                cv2.COLOR_BGR2RGB)
sign_data_process = preprocess(sign_data)

In [ ]:
visualize_class(sign_data, sign_classes, maxtitlelen=10, figsize=(20, 2))
visualize_class(sign_data_process.reshape(sign_data_process.shape[:3]), figsize=(20, 2), cmap="gray")

### Predict the Sign Type for Each Image

In [ ]:
n_preds = 5
prediction_operation = tf.nn.top_k(tf.nn.softmax(logits), n_preds)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, SAVE_FILE)
    probs, pred_classes = sess.run(prediction_operation,
                                   feed_dict={X_vals: sign_data_process, keep_prob: 1.})

In [ ]:
def prediction_confidence(img, likelihoods, predictions):
    labels = ["{:.4f}% : {}".format(likelihoods[i]*100, class_labels[str(c)])
              for i, c in enumerate(predictions)]
    y_pos = np.arange(len(labels))

    plt.figure(figsize=(4,2))
    plt.subplot(121, xticks=[], yticks=[])
    plt.imshow(img)
    plt.subplot(122)
    plt.barh(y_pos, likelihoods)
    plt.yticks(y_pos, labels, position=(1.2,1), ha="left", weight="semibold")
    plt.show()

### Output Top 5 Softmax Probabilities For Each Image Found on the Web

In [ ]:
for idx in range(n_preds):
    prediction_confidence(sign_data[idx], probs[idx], pred_classes[idx])

### Analyze Performance

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    tf.train.Saver().restore(sess, SAVE_FILE)
    new_img_accuracy = evaluate(sign_data_process, sign_classes)
    print("New Image Accuracy = {:.3f}".format(new_img_accuracy))

---

## Step 4: Visualize the Neural Network's State with Test Images

In [ ]:
def outputFeatureMap(session, image_input, tf_activation, rows=6, cols=8, figsize=(15,25), activation_min=-1, activation_max=-1 ,plt_num=1):
    """
    image_input:
        the test image being fed into the network to produce the feature maps
    tf_activation:
        should be a tf variable name used during your training procedure that
        represents the calculated state of a specific weight layer
    activation_min/max:
        can be used to view the activation contrast in more detail, by default
        matplot sets min and max to the actual min and max values of the output
    plt_num:
        used to plot out multiple different weight feature map sets on the same block,
        just extend the plt number for each new feature map entry
    """
    activation = tf_activation.eval(session=session)
    featuremaps = activation.shape[-1]
    assert rows*cols == featuremaps, "expected {}*{} to equal {}".format(rows, cols, featuremaps)
    
    print("=" * 75)
    print(tf_activation)
    print()

    plt.figure(plt_num, figsize=figsize)
    for featuremap in range(featuremaps):
        plt.subplot(rows, cols, featuremap+1)
        plt.axis("off")
        if activation_min != -1 & activation_max != -1:
            plt.imshow(activation[0,:,:,featuremap], interpolation="nearest", vmin=activation_min, vmax=activation_max, cmap="gray")
        elif activation_max != -1:
            plt.imshow(activation[0,:,:,featuremap], interpolation="nearest", vmax=activation_max, cmap="gray")
        elif activation_min !=-1:
            plt.imshow(activation[0,:,:,featuremap], interpolation="nearest", vmin=activation_min, cmap="gray")
        else:
            plt.imshow(activation[0,:,:,featuremap], interpolation="nearest")
    plt.show()

In [ ]:
conv1_weights,_ = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="conv1_1")
conv2_weights,_ = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="conv2_1")
conv3_weights,_ = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="conv3_1")

sign_img = sign_data_process[0]

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, SAVE_FILE)
    outputFeatureMap(sess, sign_img, conv1_weights, rows=4, cols=8, figsize=(5,5))
    outputFeatureMap(sess, sign_img, conv2_weights, rows=16, cols=4, plt_num=2, figsize=(5,6))
    outputFeatureMap(sess, sign_img, conv3_weights, rows=32, cols=4, plt_num=3, figsize=(9,9))